In [1]:
import sys
sys.path.append('..')
import random

from tqdm import tqdm
import numpy as np
import pandas as pd
from finetune.auto_label import *

SEED = 93
np.random.seed(SEED)
random.seed(SEED)
tqdm.pandas()

In [2]:
API_KEY_PATH = "../key"
FINETUNED_PATH = "gs://scraped-news-article-data-null/summary_human_eval.xlsx"
OG_PATH = "gs://scraped-news-article-data-null/summary_human_eval_og.xlsx"
TEMPERATURE = 0
MODEL_NAME = "gpt-3.5-turbo"

In [3]:
with open(API_KEY_PATH, "r") as fp:
    API_KEY = fp.read().strip()

finetuned_sample = pd.read_excel(FINETUNED_PATH, index_col=0)
og_sample = pd.read_excel(OG_PATH, index_col=0)
try:
    finetuned_sample = finetuned_sample.drop(["thought", "rating"], axis=1)
except KeyError:
    pass
try:
    og_sample = og_sample.drop(["thought", "rating"], axis=1)
except KeyError:
    pass
finetuned_sample.head()

,source,id,category,title,published,body,summary,summary_type,predicted
2402,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,\n* A software company is opening its own law ...
1910,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,\n* A new survey of college-bound students and...
960,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,\n* Cepsa to invest 3 bln euros in renewable e...
2348,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"\n* S&P 500 futures down 1.2%, yuan up 0.2%, w..."
4087,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,\n* The saga over Washington's embattled footb...


In [4]:
from langchain.chat_models import ChatOpenAI
import chromadb

chroma = chromadb.EphemeralClient()
collection = chroma.get_or_create_collection("temp-examples")
collection.query(query_texts=["warm-up"], n_results=1)
meal_selector = ChromaRatingExampleSelector(collection)
plan_llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0,
                          model_name=MODEL_NAME)

In [5]:
def rate_summary(row):
    text = RatableText(context_text=row["body"], output=row["predicted"])
    return evaluate_text(plan_llm, text, config.LABEL_SUMMARY_SYSTEM, config.LABEL_SUMMARY_USER)

ratings = finetuned_sample.progress_apply(rate_summary, axis=1).tolist()

100%|██████████| 10/10 [01:16<00:00,  7.66s/it]


In [6]:
rated_finetuned = pd.concat([finetuned_sample.reset_index(drop=True), pd.DataFrame(ratings).drop("raw", axis=1)], axis=1)
rated_finetuned.head()

,source,id,category,title,published,body,summary,summary_type,predicted,rating,thought
0,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,\n* A software company is opening its own law ...,4.0,The summary accurately captures the main point...
1,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,\n* A new survey of college-bound students and...,4.0,The summary accurately captures the main point...
2,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,\n* Cepsa to invest 3 bln euros in renewable e...,3.0,Thought process for the rating:\n- The summary...
3,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"\n* S&P 500 futures down 1.2%, yuan up 0.2%, w...",3.5,Thought process for the rating:\n- The summary...
4,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,\n* The saga over Washington's embattled footb...,4.0,The summary reflects the content of the text w...


In [7]:
rated_finetuned.to_excel(FINETUNED_PATH, index=False)

In [8]:
ratings = og_sample.progress_apply(rate_summary, axis=1).tolist()
rated_og = pd.concat([og_sample.reset_index(drop=True), pd.DataFrame(ratings).drop("raw", axis=1)], axis=1)
rated_og.head()

100%|██████████| 10/10 [01:31<00:00,  9.18s/it]


,source,id,category,title,published,body,summary,summary_type,predicted,rating,thought
0,reuters,22071,Software company morphs into personal injury firm,Plaintiffs' BarLegal InnovationLegal IndustryL...,2022-06-27T22:46:00,summarize in bullet points: (Reuters) - A soft...,* Mighty Law says it will undercut personal in...,BULLETS,"The company, called Mighty Group Inc., is laun...",5.0,The summary accurately captures the main point...
1,cnbc,8940,Personal Finance,College hopefuls have a new ultimate dream sch...,2023-03-15T13:11:19+00:00,summarize in bullet points: ## Massachusetts I...,"* This year, the school named by the highest n...",BULLETS,A new survey of college-bound students and the...,5.0,The summary accurately captures the main point...
2,reuters,50327,LNGFuel OilGasHydrogenRenewable Fuels,Spain's Cepsa to invest 3 bln euros in green h...,2022-12-01T15:03:00,"summarize in bullet points: CADIZ, Spain, Dec ...","* To produce 300,000 tons of green hydrogen pe...",BULLETS,"The project will produce 300,000 tons of hydro...",2.0,The summary accurately mentions that the proje...
3,reuters,22786,U.S. Markets,"Wall St falls after recent strong gains, Alpha...",2023-02-09T00:30:00,"summarize in bullet points: NEW YORK, Feb 8 (R...",* Alphabet shares dive after Google AI chatbot...,BULLETS,"U.S. stocks ended lower on Wednesday, weighed ...",1.0,The summary does not accurately reflect the co...
4,cnbc,1864,Sports,Canadian billionaire Steve Apostolopoulos says...,2023-04-24T17:42:10+00:00,summarize in bullet points: Canadian billionai...,* Canadian businessman Steve Apostolopoulos sa...,BULLETS,The saga over Washington's embattled football ...,5.0,Thought process for the rating:\n- The summary...


In [9]:
rated_og.to_excel(OG_PATH, index=False)

In [10]:
rated_finetuned.rating.describe()

count    10.000000
mean      4.100000
std       0.614636
min       3.000000
25%       4.000000
50%       4.000000
75%       4.375000
max       5.000000
Name: rating, dtype: float64

In [11]:
rated_og.rating.describe()

count    10.000000
mean      3.600000
std       1.429841
min       1.000000
25%       2.500000
50%       4.000000
75%       4.750000
max       5.000000
Name: rating, dtype: float64